In [3]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [ ]:
# Load min-temp and max-temp data from multiple cities
# Turn all of that data into a single data frame with state, city, date, min, and max temp

import glob

all_dfs = [] 

for one_filename in glob.glob('../data/*,*.csv'): 
    print(f'Loading {one_filename}...')

    city, state = (
        one_filename
        .removeprefix('../data/')
        .removesuffix('.csv')
        .split(',')
    )

    one_df = (
        pd
        .read_csv(one_filename,
                  usecols=[0, 1, 2], 
                  names=['date_time',
                         'max_temp',
                         'min_temp'], 
                  header=0) 
        .assign(city=city.replace('+', ' ').title(), 
                state=state.upper()) 
    )

    all_dfs.append(one_df) 

df = pd.concat(all_dfs)


Loading ../data/san+francisco,ca.csv...
Loading ../data/new+york,ny.csv...
Loading ../data/springfield,ma.csv...
Loading ../data/boston,ma.csv...
Loading ../data/springfield,il.csv...
Loading ../data/albany,ny.csv...
Loading ../data/los+angeles,ca.csv...
Loading ../data/chicago,il.csv...


In [9]:
df.groupby(['state', 'city'])['date_time'].min().sort_values()

state  city         
CA     Los Angeles      2018-12-11 00:00:00
       San Francisco    2018-12-11 00:00:00
IL     Chicago          2018-12-11 00:00:00
       Springfield      2018-12-11 00:00:00
MA     Boston           2018-12-11 00:00:00
       Springfield      2018-12-11 00:00:00
NY     Albany           2018-12-11 00:00:00
       New York         2018-12-11 00:00:00
Name: date_time, dtype: object

In [10]:
df.groupby(['state', 'city'])['date_time'].max().sort_values()

state  city         
CA     Los Angeles      2019-03-11 21:00:00
       San Francisco    2019-03-11 21:00:00
IL     Chicago          2019-03-11 21:00:00
       Springfield      2019-03-11 21:00:00
MA     Boston           2019-03-11 21:00:00
       Springfield      2019-03-11 21:00:00
NY     Albany           2019-03-11 21:00:00
       New York         2019-03-11 21:00:00
Name: date_time, dtype: object

In [11]:
df.groupby(['state', 'city'])['date_time'].agg(['min', 'max'])

min                  max
state city                                                   
CA    Los Angeles    2018-12-11 00:00:00  2019-03-11 21:00:00
      San Francisco  2018-12-11 00:00:00  2019-03-11 21:00:00
IL    Chicago        2018-12-11 00:00:00  2019-03-11 21:00:00
      Springfield    2018-12-11 00:00:00  2019-03-11 21:00:00
MA    Boston         2018-12-11 00:00:00  2019-03-11 21:00:00
      Springfield    2018-12-11 00:00:00  2019-03-11 21:00:00
NY    Albany         2018-12-11 00:00:00  2019-03-11 21:00:00
      New York       2018-12-11 00:00:00  2019-03-11 21:00:00

In [12]:
df.groupby(['state', 'city'])['min_temp'].min()

state  city         
CA     Los Angeles       4
       San Francisco     3
IL     Chicago         -28
       Springfield     -25
MA     Boston          -14
       Springfield     -20
NY     Albany          -19
       New York        -14
Name: min_temp, dtype: int64

In [13]:
# Grouping by state-city combinations, get the min and max temperatures
# Then apply the `describe` method, which returns a data frame
df.groupby(['state', 'city'])[['min_temp', 'max_temp']].apply(DataFrame.describe)

min_temp    max_temp
state city                                     
CA    Los Angeles count  728.000000  728.000000
                  mean    10.637363   17.054945
                  std      2.705200    2.708640
                  min      4.000000   12.000000
                  25%      9.000000   15.000000
...                             ...         ...
NY    New York    min    -14.000000  -12.000000
                  25%     -4.000000    2.000000
                  50%      0.000000    4.000000
                  75%      2.000000    7.000000
                  max     12.000000   15.000000

[64 rows x 2 columns]

In [14]:
import glob

all_dfs = []

for one_filename in glob.glob('../data/*,*.csv'):
    print(f'Loading {one_filename}...')
    city, state = one_filename.removeprefix('../data/').removesuffix('.csv').split(',')
    one_df = pd.read_csv(one_filename, 
                     usecols=[1, 2, 19],
                     names=['max_temp', 'min_temp', 'precipMM'],
                     header=0)
    one_df['city'] = city.replace('+', ' ').title()
    one_df['state'] = state.upper()
    all_dfs.append(one_df)

df = pd.concat(all_dfs)
df.head()

Loading ../data/san+francisco,ca.csv...
Loading ../data/new+york,ny.csv...
Loading ../data/springfield,ma.csv...
Loading ../data/boston,ma.csv...
Loading ../data/springfield,il.csv...
Loading ../data/albany,ny.csv...
Loading ../data/los+angeles,ca.csv...
Loading ../data/chicago,il.csv...


,max_temp,min_temp,precipMM,city,state
0,13,8,0.0,San Francisco,CA
1,13,8,0.0,San Francisco,CA
2,13,8,0.0,San Francisco,CA
3,13,8,0.0,San Francisco,CA
4,13,8,0.0,San Francisco,CA


In [15]:
# Which cities had, on at least 3 occasions, precipitation of 15 mm or more?

# Technique 1: Using a function without extra arguments

def has_multiple_readings_at_least(mini_df):
    return mini_df.loc[
        mini_df['precipMM'] >= 15,
        'precipMM'
    ].count() >= 3

(
    df
    .groupby(['city', 'state'])
    .filter(has_multiple_readings_at_least)
    [['city', 'state']]
    .drop_duplicates()
)

,city,state
0,New York,NY
0,Boston,MA
0,Los Angeles,CA


In [16]:
# Technique 2: Using a function with extra arguments

def has_multiple_readings_at_least(mini_df, min_mm, times):
    return mini_df.loc[
        mini_df['precipMM'] >= min_mm,
        'precipMM'
        ].count() >= times

(
    df
    .groupby(['city', 'state'])
    .filter(has_multiple_readings_at_least,
            min_mm=15,
            times=3)
    [['city', 'state']]
    .drop_duplicates()
)

,city,state
0,New York,NY
0,Boston,MA
0,Los Angeles,CA


In [17]:

# Find cities that had at least 3 measurements of 10 mm precipitation or more, when the temperature was below 0 Celsuius.

def has_multiple_readings_at_least(mini_df, min_mm, times):
    return mini_df.loc[
        ((mini_df['precipMM'] >= min_mm) &
         (mini_df['min_temp'] <= 0)),
        'precipMM'
        ].count() >= times

(
    df
    .groupby(['city', 'state'])
    .filter(has_multiple_readings_at_least, min_mm=10, times=3)
    [['city', 'state']]
    .drop_duplicates()
)

,city,state
0,New York,NY
0,Boston,MA
0,Albany,NY


In [18]:
# For each precipitation measurement, calculate the proportion of that city's total precipitation.
def proportion_of_city_precip(s):
    return s / s.sum()

df['precip_pct'] = (
    df
    .groupby('city')['precipMM']
    .transform(proportion_of_city_precip)
)

In [19]:
df.groupby(['city', 'state'])['precip_pct'].max()

city           state
Albany         NY       0.029228
Boston         MA       0.048302
Chicago        IL       0.057257
Los Angeles    CA       0.059242
New York       NY       0.055149
San Francisco  CA       0.056509
Springfield    IL       0.030977
               MA       0.023459
Name: precip_pct, dtype: float64